#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

# Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [29]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
!pip install folium

Распакуем наши данные из архива. (upd: я архив сам распаковал)

In [15]:
!unzip /content/sample_data/archive.zip -d /content/sample_data/

unzip:  cannot find or open /content/sample_data/archive.zip, /content/sample_data/archive.zip.zip or /content/sample_data/archive.zip.ZIP.


Загрузим данные из csv файла в датафрейм.

In [70]:
dataframe = pd.read_csv('/content/sample_data/input_data.csv', sep=';')

dataframe

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0.0,2.0,620000.0,NaN,66.0,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0.0,0.0,385000.0,NaN,1.0,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3.0,0.0,142701.0,242543.0,50.0,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0.0,0.0,385000.0,NaN,1.0,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3.0,2.0,353960.0,439378.0,23.0,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335562,2021-02-23,2220000,3,5,3,58.2,6.0,57.919796,59.968609,0.0,0.0,622036.0,NaN,66.0,NaN
1335563,2021-02-23,430000,1,2,2,39.0,-100.0,58.672893,52.179821,6.0,0.0,612740.0,NaN,43.0,NaN
1335564,2021-02-23,3400000,6,12,1,43.8,0.0,57.098044,65.617530,2.0,0.0,625053.0,450441.0,72.0,1882914.0
1335565,2021-02-23,3360000,1,6,1,42.0,0.0,45.025520,39.021055,0.0,0.0,350075.0,NaN,23.0,NaN


Отобразим на карте координаты наших построек.



In [71]:
import folium
from IPython.display import display

map_df = dataframe.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [72]:
# Преобразуем дату в формат datetime и затем в категориальный признак
dataframe['date'] = pd.to_datetime(dataframe['date'], errors='coerce')
dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')  # Преобразование в строку формата 'YYYY-MM-DD'
dataframe['date'] = dataframe['date'].astype('category')  # Преобразование в категориальный тип

# Определяем координаты центров городов
moscow_coords = np.array([55.7558, 37.6173])
spb_coords = np.array([59.9342, 30.3350])

# Функция для вычисления расстояния по формуле Хаверсина
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # радиус Земли в километрах
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat / 2) ** 2 +
         np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Преобразуем столбцы координат в числовой формат
dataframe['geo_lat'] = pd.to_numeric(dataframe['geo_lat'], errors='coerce')
dataframe['geo_lon'] = pd.to_numeric(dataframe['geo_lon'], errors='coerce')

# Создаем массивы с координатами
latitudes = dataframe['geo_lat'].values
longitudes = dataframe['geo_lon'].values

# Вычисляем расстояния до Москвы и Санкт-Петербурга
distances_to_moscow = haversine_distance(latitudes, longitudes, moscow_coords[0], moscow_coords[1])
distances_to_spb = haversine_distance(latitudes, longitudes, spb_coords[0], spb_coords[1])

# Создаем признаки is_Moscow и is_Saint_Peterburg
dataframe['is_Moscow'] = distances_to_moscow <= 20
dataframe['is_Saint_Peterburg'] = distances_to_spb <= 20

# Проверка добавленных признаков
dataframe.head(20)

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.30,0.0,56.780112,60.699355,0.0,2.0,620000.0,NaN,66.0,1632918.0,False,False
1,2021-01-01,1450000,5,5,1,33.00,6.0,44.608154,40.138381,0.0,0.0,385000.0,NaN,1.0,NaN,False,False
2,2021-01-01,10700000,4,13,3,85.00,12.0,55.540060,37.725112,3.0,0.0,142701.0,242543.0,50.0,681306.0,False,False
3,2021-01-01,3100000,3,5,3,82.00,9.0,44.608154,40.138381,0.0,0.0,385000.0,NaN,1.0,NaN,False,False
4,2021-01-01,2500000,2,3,1,30.00,9.0,44.738685,37.713668,3.0,2.0,353960.0,439378.0,23.0,1730985.0,False,False
5,2021-01-01,1450000,5,5,2,47.00,6.0,48.511172,44.566846,2.0,0.0,400096.0,260588.0,34.0,1009994.0,False,False
6,2021-01-01,9000000,2,4,3,107.40,21.3,55.009914,82.934859,4.0,0.0,630102.0,233285.0,54.0,2823596.0,False,False
7,2021-01-01,2990000,1,2,3,54.00,7.0,51.834703,107.600571,0.0,0.0,670034.0,NaN,3.0,NaN,False,False
8,2021-01-01,2300000,16,18,1,39.70,11.5,45.003869,39.086511,4.0,0.0,350065.0,523822.0,23.0,1284243.0,False,False
9,2021-01-01,2290000,2,2,2,53.20,16.0,53.164362,45.033956,5.0,0.0,440003.0,NaN,58.0,NaN,False,False


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [73]:
dataframe.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], inplace=True)

dataframe

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0.0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0.0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3.0,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0.0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3.0,False,False
...,...,...,...,...,...,...,...,...,...,...
1335562,2021-02-23,2220000,3,5,3,58.2,6.0,0.0,False,False
1335563,2021-02-23,430000,1,2,2,39.0,-100.0,6.0,False,False
1335564,2021-02-23,3400000,6,12,1,43.8,0.0,2.0,False,False
1335565,2021-02-23,3360000,1,6,1,42.0,0.0,0.0,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: (Ваш ответ)

Числовые: (Ваш ответ)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [75]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


en = OneHotEncoder(drop='first')
ended = en.fit_transform(dataframe[['building_type', 'is_Moscow', 'is_Saint_Peterburg']])

ended_dataframe = pd.DataFrame(ended.toarray(), columns=en.get_feature_names_out(['building_type', 'is_Moscow', 'is_Saint_Peterburg']))

dataframe_ended = pd.concat([dataframe.drop(columns=['building_type', 'is_Moscow', 'is_Saint_Peterburg']), ended_dataframe], axis=1)

dataframe_ended.head(10)

,date,price,level,levels,rooms,area,kitchen_area,building_type_1.0,building_type_2.0,building_type_3.0,building_type_4.0,building_type_5.0,building_type_6.0,building_type_nan,is_Moscow_True,is_Saint_Peterburg_True
0,2021-01-01,2451300,15,31,1,30.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2021-01-01,1450000,5,5,2,47.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2021-01-01,9000000,2,4,3,107.4,21.3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,2021-01-01,2990000,1,2,3,54.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021-01-01,2300000,16,18,1,39.7,11.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,2021-01-01,2290000,2,2,2,53.2,16.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [78]:
from sklearn.preprocessing import StandardScaler

#dataframe['date'] = pd.to_datetime(dataframe['date'].astype(str))
#dataframe['lev_dif'] = np.where(dataframe['levels'] == 0, 0, dataframe['level'] / dataframe['levels'])
#dataframe['days_dif'] = (dataframe['date'] - dataframe['date'].min()).dt.days
#dataframe = dataframe.drop(columns=['date'])

scaler = StandardScaler()
dataframe[['level', 'levels', 'area', 'rooms', 'kitchen_area', 'days_dif', 'lev_dif']] = scaler.fit_transform(dataframe[['level', 'levels', 'area', 'rooms', 'kitchen_area', 'days_dif', 'lev_dif']])

dataframe

,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,lev_dif,days_dif
0,2451300,1.534178,2.546375,-0.624605,-0.855058,0.122999,0.0,False,False,-0.301733,-2.232788
1,1450000,-0.289315,-0.941523,-0.624605,-0.755623,0.298387,0.0,False,False,1.382902,-2.232788
2,10700000,-0.471665,0.131677,1.099295,1.159429,0.473776,3.0,False,False,-0.876777,-2.232788
3,3100000,-0.654014,-0.941523,1.099295,1.048945,0.386081,0.0,False,False,0.077310,-2.232788
4,2500000,-0.836363,-1.209822,-0.624605,-0.866106,0.386081,3.0,False,False,0.294908,-2.232788
...,...,...,...,...,...,...,...,...,...,...,...
1335562,2220000,-0.654014,-0.941523,1.099295,0.172441,0.298387,0.0,False,False,0.077310,1.517699
1335563,430000,-1.018713,-1.343972,0.237345,-0.534655,-2.800142,6.0,False,False,-0.249088,1.517699
1335564,3400000,-0.106966,-0.002473,-0.624605,-0.357881,0.122999,2.0,False,False,-0.249088,1.517699
1335565,3360000,-1.018713,-0.807373,-0.624605,-0.424172,0.122999,0.0,False,False,-1.337081,1.517699


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.





In [79]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x_test in X.values:
            distances = np.sqrt(np.sum((self.X_train.values - x_test) ** 2, axis=1))
            nearest_neighbors_indices = np.argsort(distances)[:self.n_neighbors]
            neighbor_prices = self.y_train.values[nearest_neighbors_indices]
            predictions.append(np.mean(neighbor_prices))
        return np.array(predictions)

    def score(self, X, y):
        predictions = self.predict(X)
        rmse = np.sqrt(np.mean((predictions - y) ** 2))
        return rmse

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [80]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Инициализация дополнительных переменных для Momentum и AdaGrad
        if self.optimization == 'Momentum':
            self.velocity_w = np.zeros(n_features)
            self.velocity_b = 0
        elif self.optimization == 'AdaGrad':
            self.accumulated_gradient_w = np.zeros(n_features)
            self.accumulated_gradient_b = 0

        # Обучение
        for _ in range(self.max_iter):
            # Прогноз
            y_predicted = np.dot(X, self.weights) + self.bias

            # Вычисляем градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновление параметров в зависимости от выбранного метода
            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            elif self.optimization == 'Momentum':
                self.velocity_w = self.decay_rate * self.velocity_w + (1 - self.decay_rate) * dw
                self.velocity_b = self.decay_rate * self.velocity_b + (1 - self.decay_rate) * db
                self.weights -= self.learning_rate * self.velocity_w
                self.bias -= self.learning_rate * self.velocity_b
            elif self.optimization == 'AdaGrad':
                self.accumulated_gradient_w += dw ** 2
                self.accumulated_gradient_b += db ** 2
                adjusted_lr_w = self.learning_rate / (np.sqrt(self.accumulated_gradient_w) + self.epsilon)
                adjusted_lr_b = self.learning_rate / (np.sqrt(self.accumulated_gradient_b) + self.epsilon)
                self.weights -= adjusted_lr_w * dw
                self.bias -= adjusted_lr_b * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def score(self, X, y):
        y_predicted = self.predict(X)
        mse = np.mean((y_predicted - y) ** 2)
        return mse

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [82]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Создайте и разделите набор данных
X, y = make_regression(n_samples=1000, n_features=5, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ваши классы
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Инициализация дополнительных переменных для Momentum и AdaGrad
        if self.optimization == 'Momentum':
            self.velocity_w = np.zeros(n_features)
            self.velocity_b = 0
        elif self.optimization == 'AdaGrad':
            self.accumulated_gradient_w = np.zeros(n_features)
            self.accumulated_gradient_b = 0

        # Обучение
        for _ in range(self.max_iter):
            # Прогноз
            y_predicted = np.dot(X, self.weights) + self.bias

            # Вычисляем градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновление параметров в зависимости от выбранного метода
            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            elif self.optimization == 'Momentum':
                self.velocity_w = self.decay_rate * self.velocity_w + (1 - self.decay_rate) * dw
                self.velocity_b = self.decay_rate * self.velocity_b + (1 - self.decay_rate) * db
                self.weights -= self.learning_rate * self.velocity_w
                self.bias -= self.learning_rate * self.velocity_b
            elif self.optimization == 'AdaGrad':
                self.accumulated_gradient_w += dw ** 2
                self.accumulated_gradient_b += db ** 2
                adjusted_lr_w = self.learning_rate / (np.sqrt(self.accumulated_gradient_w) + self.epsilon)
                adjusted_lr_b = self.learning_rate / (np.sqrt(self.accumulated_gradient_b) + self.epsilon)
                self.weights -= adjusted_lr_w * dw
                self.bias -= adjusted_lr_b * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def score(self, X, y):
        y_predicted = self.predict(X)
        mse = np.mean((y_predicted - y) ** 2)
        return mse

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x_test in X:
            distances = np.sqrt(np.sum((self.X_train - x_test) ** 2, axis=1))
            nearest_neighbors_indices = np.argsort(distances)[:self.n_neighbors]
            neighbor_prices = self.y_train[nearest_neighbors_indices]
            predictions.append(np.mean(neighbor_prices))
        return np.array(predictions)

    def score(self, X, y):
        predictions = self.predict(X)
        rmse = np.sqrt(np.mean((predictions - y) ** 2))
        return rmse

# Обучаем вашу линейную регрессию
linear_regressor = LinearRegression(learning_rate=0.01, max_iter=1000)
linear_regressor.fit(X_train, y_train)
y_pred_linear = linear_regressor.predict(X_test)

# Оценка модели
mse_linear = mean_squared_error(y_test, y_pred_linear)
mae_linear = mean_absolute_error(y_test, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)

print("Linear Regression (Custom Implementation):")
print(f"MSE: {mse_linear}, MAE: {mae_linear}, RMSE: {rmse_linear}")

# Обучаем библиотечную линейную регрессию
from sklearn.linear_model import LinearRegression as SklearnLinearRegression
from sklearn.neighbors import KNeighborsRegressor as SklearnKNNRegressor

sklearn_linear_regressor = SklearnLinearRegression()
sklearn_linear_regressor.fit(X_train, y_train)
y_pred_sklearn_linear = sklearn_linear_regressor.predict(X_test)

# Оценка модели
mse_sklearn_linear = mean_squared_error(y_test, y_pred_sklearn_linear)
mae_sklearn_linear = mean_absolute_error(y_test, y_pred_sklearn_linear)
rmse_sklearn_linear = np.sqrt(mse_sklearn_linear)

print("\nLinear Regression (Sklearn Implementation):")
print(f"MSE: {mse_sklearn_linear}, MAE: {mae_sklearn_linear}, RMSE: {rmse_sklearn_linear}")



Linear Regression (Custom Implementation):
MSE: 0.01119622755970962, MAE: 0.08557795400845804, RMSE: 0.10581222783643496

Linear Regression (Sklearn Implementation):
MSE: 0.011090609683464713, MAE: 0.08535103887972018, RMSE: 0.1053119636293271
